In [2]:
!pip install gurobipy  # install gurobipy, if not already installed

     |████████████████████████████████| 11.5 MB 18.5 MB/s 


In [3]:
from ast import Pass
from re import M
from turtle import shape
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [4]:
def complete_graph(graph):
        graph_original = graph
        for e in graph_original.edges():
            graph_original[e[0]][e[1]]['length'] = int(graph_original[e[0]][e[1]]['length'])
        graph_complete = graph_original.copy()
        nodes = list(graph_original.nodes())
        paths = {}
        paths = dict(nx.all_pairs_dijkstra_path(graph_original, weight = 'length'))
        for i in nodes:
            for j in nodes:
                if i != j and not (i, j) in graph_complete.edges():
                    graph_complete.add_edge(i, j)
                    graph_complete[i][j]['name'] = '{}to{}'.format(i, j)
                    graph_complete[i][j]['length'] = 0
                    paths[(i, j)] = paths[i][j]
                    for k in range(len(paths[i, j]) - 1):
                        graph_complete[i][j]['length'] += graph_original[paths[i][j][k]][paths[i][j][k + 1]]['length']
                elif i != j:
                    paths[(i, j)] = [i, j]

        return graph_complete

In [ ]:
temp = np.zeros(shape=(5))
temp[0]

0.0

In [5]:
class mid_mile:

    def __init__(self, K, L, M):

        #self.graph = graph
        self.K = K
        self.L = L                                       # no of legs
        self.M = M
        self.V = np.zeros(shape=(self.K))           # the demand of each commodity K in route R
        self.v = np.zeros(shape=(self.L, self.M))
        self.C = None                                         
        self.F = np.zeros(shape=(self.L, self.M))
        self.A = np.zeros(shape=(self.L, self.M))
        self.B = np.zeros(shape=(self.L, self.M))
        self.Qmax = np.zeros(shape=(self.L, self.M))
        self.Qmin = np.zeros(shape=(self.L, self.M))

        self.model = gp.Model()
        
    def add_variables(self):
        lm = []

        # decision variables: xr, ylm, vlm, flm
        for a in range(self.L):
            for b in range(self.M):
                lm.append((a,b))

        lm = gp.tuplelist(lm)

        self.y = self.model.addVars(lm, vtype=GRB.BINARY, name="y")
        self.f = self.model.addVars(lm, vtype=GRB.INTEGER, name="f")

        self.model.update()

    def add_init_constraints(self):
        self.add_constraints1()
        self.add_constraints2()
        self.add_constraints3()
        self.add_constraints4()
        self.add_constraints5()
        #self.obj_constraint()

    def add_constraints1(self):
        pass

    def add_constraints2(self):
        # summation of shipments should tally
        
        lhs = 0
        for k in range(self.K):
            lhs += self.V[k]
        
        print("--- Constraint 2 running: lhs = {} ---".format(lhs))
        
        rhs = 0
        for l in range(self.L):
            for m in range(self.M):
                rhs += self.v[l][m] * self.f[l,m]
            
        self.model.addConstr(lhs - rhs <= 0)
        
        self.model.update()

    def add_constraints3(self):
        pass

    def add_constraints4(self):
        pass
    
    def add_constraints5(self):
        for l in range(self.L):
            for m in range(self.M):
                lhs = self.f[l, m]
                rhs = self.F[l][m] * self.y[l, m]
                self.model.addConstr(lhs - rhs <= 0)
        
        self.model.update()


    # def obj_constraint(self):
    #     lhs = 0
    #     for e in self.edges:
    #         lhs += self.graph[e[0]][e[1]]['length'] * self.x[a, e[0], e[1]]
    #     #this constraint ensures that c is atleast equal to the max of the walks of the agents
    #     print(self.c)
    #     self.model.addConstr(lhs <= self.c[0], name="obj_agent{}".format(a))
        
    #     self.model.update()


    def objective(self):
        term2 = 0
        for l in range(self.L):
            for m in range(self.M):
                term2 += self.A[l][m]*self.f[l,m]
        
        obj = term2

        self.model.setObjective(obj, GRB.MINIMIZE)

    def process(self):
        self.status = self.model.optimize()

    def get_solution(self):
        Pass

In [6]:
# Input model parameter values

K = 1

L = 3
M = 1                       # only one mode of transport

V = np.zeros(shape=(K))
V[0] = 10

v = np.zeros(shape=(L, M))
v[0][0] = 1
v[1][0] = 1
v[2][0] = 1


F = np.zeros(shape=(L, M))
F[0][0] = 3
F[1][0] = 7
F[2][0] = 4

A = np.zeros(shape=(L, M))
A[0][0] = 5
A[1][0] = 10
A[2][0] = 1


# F = np.zeros(shape=(L, M))
# for i in range(L):
#     for j in range(M):
#         F[i][j] = 5*i + j

# A = np.zeros(shape=(L, M))
# for i in range(L):
#     for j in range(M):
#         A[i][j] = 2*(i+1) + 3*(j+1)

In [ ]:
print(V)

[10.]


In [7]:
test = mid_mile(K, L, M)

test.V = V
test.v = v
test.F = F
test.A = A

Restricted license - for non-production use only - expires 2023-10-25


In [8]:
test.add_variables()
test.add_init_constraints()
test.objective()
test.process()

--- Constraint 2 running: lhs = 10.0 ---
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 4 rows, 6 columns and 9 nonzeros
Model fingerprint: 0xf46a3b4b
Variable types: 0 continuous, 6 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 4 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 49 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.900000000000e+01, best bound 4.900000000000e+01, gap 0.0000%


In [ ]:
y_sol = test.model.getAttr('X', test.y)
f_sol = test.model.getAttr('X', test.f)
print(f_sol)

{(0, 0): 3.0, (1, 0): 3.0, (2, 0): 4.0}
